In [176]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Normalization
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten
from keras import layers

In [260]:
df_merged.index

MultiIndex([('2021-05-26',    'AFD'),
            ('2021-05-26',    'CDU'),
            ('2021-05-26',    'FDP'),
            ('2021-05-26', 'GRUENE'),
            ('2021-05-26',  'LINKE'),
            ('2021-05-26',    'SPD'),
            ('2021-05-26',  'other'),
            ('2021-05-27',    'AFD'),
            ('2021-05-27',    'CDU'),
            ('2021-05-27',    'FDP'),
            ...
            ('2021-08-21',  'LINKE'),
            ('2021-08-21',    'SPD'),
            ('2021-08-21',  'other'),
            ('2021-08-22',    'AFD'),
            ('2021-08-22',    'CDU'),
            ('2021-08-22',    'FDP'),
            ('2021-08-22', 'GRUENE'),
            ('2021-08-22',  'LINKE'),
            ('2021-08-22',    'SPD'),
            ('2021-08-22',  'other')],
           names=['created_at', 'party'], length=623)

In [177]:
df_merged = pd.read_csv("df_merged.csv")

In [178]:
df_merged["created_at"] = pd.to_datetime(df_merged["created_at"])

In [179]:
df_merged = df_merged.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "replies_count": "sum", 
    "retweets_count": "sum",
    "likes_count": "sum",
    "avg_len_of_tweet": "mean" ,
    "share_of_tweets" :"mean",
    "share_unique_users": "mean",
    "weighted_sentiment" : "mean",
    "share_of_positive_tweets": "mean",
    "share_of_negative_tweets" : "mean",
    "Polls": "mean"}) 

In [192]:
df_merged.iloc[1]

replies_count                0.0
retweets_count               0.0
likes_count                  0.0
avg_len_of_tweet             NaN
share_of_tweets              NaN
share_unique_users           NaN
weighted_sentiment           NaN
share_of_positive_tweets     NaN
share_of_negative_tweets     NaN
Polls                       25.5
Name: (2021-05-26 00:00:00, CDU), dtype: float64

In [225]:
df_merged = df_merged.replace(np.nan, -9999)

In [227]:
model = Sequential()
model.add(layers.Masking(mask_value=-9999., input_shape=(7,10)))
model.add(layers.LSTM(units=20, activation='tanh', return_sequences=True))
model.add(layers.LSTM(units=20, activation='tanh', return_sequences=False))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(7, activation='softmax'))


In [240]:
df_merged_test = df_merged.iloc[616:623]

In [244]:
df_merged_test_numpy = df_merged_test.to_numpy().reshape(1,7,10)

In [241]:
df_merged = df_merged.iloc[0:623]

In [272]:
df_merged.tail(50)

replies_count  retweets_count  likes_count  \
created_at party                                                
2021-08-15 other             0.0             0.0          0.0   
2021-08-16 AFD            7821.0         16054.0      94816.0   
           CDU           30584.0         40776.0     351880.0   
           FDP            3179.0          3012.0      29618.0   
           GRUENE         5554.0          8736.0      59198.0   
           LINKE          1201.0          1413.0       9016.0   
           SPD            3566.0          8864.0      54360.0   
           other             0.0             0.0          0.0   
2021-08-17 AFD            7224.0         16655.0      91083.0   
           CDU           11423.0         11373.0     104674.0   
           FDP            1489.0           958.0       8182.0   
           GRUENE         3801.0          4439.0      41800.0   
           LINKE          1360.0          1709.0       8872.0   
           SPD            3949.0          5370.0      32040.0   
           other             0.0             0.0          0.0   
2021-08-18 AFD            6488.0         12879.0      73847.0   
           CDU           12106.0         17459.0     116347.0   
           FDP            1510.0          1925.0      13444.0   
           GRUENE         4254.0          4293.0      27632.0   
           LINKE          1359.0          1073.0       7685.0   
           SPD            5339.0          9002.0      55124.0   
           other             0.0             0.0          0.0   
2021-08-19 AFD           17887.0         31569.0     191927.0   
           CDU            9004.0         11633.0      93015.0   
           FDP            1727.0          1222.0      11124.0   
           GRUENE         6249.0          7946.0      61616.0   
           LINKE          1099.0          1633.0       8407.0   
           SPD            3840.0          5417.0      38124.0   
           other             0.0             0.0          0.0   
2021-08-20 AFD           12249.0         17768.0     101640.0   
           CDU           15183.0         19001.0     152565.0   
           FDP            7265.0          3711.0      59321.0   
           GRUENE         8632.0          9330.0      69674.0   
           LINKE           813.0          1305.0       6908.0   
           SPD            4592.0          5427.0      51127.0   
           other             0.0             0.0          0.0   
2021-08-21 AFD            5801.0         12270.0      67105.0   
           CDU           36219.0         34297.0     291497.0   
           FDP            1329.0          1252.0      10686.0   
           GRUENE         4038.0          3282.0      26306.0   
           LINKE           740.0           651.0       4053.0   
           SPD            3560.0          5346.0      33256.0   
           other             0.0             0.0          0.0   
2021-08-22 AFD            7160.0         15148.0      75008.0   
           CDU           23978.0         38584.0     307038.0   
           FDP            1979.0          1700.0      15553.0   
           GRUENE         9852.0         10488.0      80533.0   
           LINKE          1470.0          1443.0       7590.0   
           SPD            6824.0          6573.0      48056.0   
           other             0.0             0.0          0.0   

                   avg_len_of_tweet  share_of_tweets  share_unique_users  \
created_at party                                                           
2021-08-15 other       -9999.000000     -9999.000000        -9999.000000   
2021-08-16 AFD           181.584823         0.115129            0.607530   
           CDU           174.399994         0.574684            0.325595   
           FDP           183.132449         0.095379            0.579310   
           GRUENE        189.386098         0.102395            0.567287   
           LINKE         189.428305         0.027172            0.610180   
           SPD           192.0

In [229]:
df_merged_numpy = df_merged.to_numpy().reshape(89,7,10)

In [230]:
y = df_merged["Polls"]

In [231]:
y_numpy = y.to_numpy().reshape(89,7)

In [266]:


model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.fit(df_merged_numpy, y_numpy ,
              epochs=30, 
              batch_size=1, 
              verbose=1, 
              validation_split=0.2)



Epoch 1/30
71/71 [==============================] - 5s 25ms/step - loss: 2432.1904 - accuracy: 0.0563 - val_loss: 2209.8279 - val_accuracy: 0.0556
Epoch 2/30
71/71 [==============================] - 1s 8ms/step - loss: 2530.5457 - accuracy: 0.1268 - val_loss: 2412.1582 - val_accuracy: 0.0556
Epoch 3/30
71/71 [==============================] - 1s 8ms/step - loss: 2676.2346 - accuracy: 0.0986 - val_loss: 2601.9893 - val_accuracy: 0.0556
Epoch 4/30
71/71 [==============================] - 1s 8ms/step - loss: 3091.0024 - accuracy: 0.0986 - val_loss: 2806.1577 - val_accuracy: 0.0556
Epoch 5/30
71/71 [==============================] - 1s 8ms/step - loss: 3046.2891 - accuracy: 0.1408 - val_loss: 2984.7378 - val_accuracy: 0.0556
Epoch 6/30
71/71 [==============================] - 1s 8ms/step - loss: 3505.3660 - accuracy: 0.1549 - val_loss: 3203.6057 - val_accuracy: 0.0556
Epoch 7/30
71/71 [==============================] - 1s 9ms/step - loss: 3554.6548 - accuracy: 0.1549 - val_loss: 3369.4478 

In [267]:
pd.DataFrame(model.predict(df_merged_test_numpy)).rename( \
                        columns = {0:'AFD',1:'CDU',2:'FDP',3: 'GRUENE',4: 'LINKE', 5:'SPD', 6: 'other'}).round(2)

,AFD,CDU,FDP,GRUENE,LINKE,SPD,other
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
